In [2]:
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import timedelta
import re
from re import match
import getpass
import io
import os, json
import netmiko
from netmiko import ConnectHandler
from py2neo import Graph

In [20]:
graph = Graph("bolt://localhost:7687", user="neo4j", password="temp",name="diagnos")

In [35]:
try:
    graph.run("Match () Return 1 Limit 1")
    print('ok')
except Exception:
    print('not ok')
my_node = graph.run("MATCH (n) DETACH DELETE n")

ok


In [34]:
diagnos = pd.read_csv (r'C:\Users\VBAPHCColleD\Downloads\diagnos.csv',skip_blank_lines=True) 
print(len(diagnos.index))
diagnos = diagnos.groupby(['Disease']).agg(tuple).applymap(list).reset_index()
print(len(diagnos.index))
diagnos["Count of Disease Occurrence"] = diagnos["Count of Disease Occurrence"].apply(lambda x: [i for i in x if str(i) != "nan"])

diagnos=diagnos.explode('Symptom')

for i, row in diagnos.iterrows():
    row['Count of Disease Occurrence']=row['Count of Disease Occurrence'][0]
#diagnos.to_csv("diag.csv",index=False)
print(diagnos)

1866
134
                                               Disease  \
0    UMLS:C0001175_acquired immuno-deficiency syndr...   
0    UMLS:C0001175_acquired immuno-deficiency syndr...   
0    UMLS:C0001175_acquired immuno-deficiency syndr...   
0    UMLS:C0001175_acquired immuno-deficiency syndr...   
0    UMLS:C0001175_acquired immuno-deficiency syndr...   
..                                                 ...   
133                            UMLS:C1623038_cirrhosis   
133                            UMLS:C1623038_cirrhosis   
133                            UMLS:C1623038_cirrhosis   
133                            UMLS:C1623038_cirrhosis   
133                            UMLS:C1623038_cirrhosis   

    Count of Disease Occurrence  \
0                         350.0   
0                         350.0   
0                         350.0   
0                         350.0   
0                         350.0   
..                          ...   
133                       218.0   
133           

In [36]:
try:
    graph.run("Match () Return 1 Limit 1")
    print('ok')
except Exception:
    print('not ok')
tx = graph.begin()
for index, row in diagnos.iterrows():
    tx.evaluate('''
    MERGE (s:SYMPTOM {Name: $Symptom})
    MERGE (d:DISEASE {Name: $Disease})
    CREATE (s)-[i:SYMPTOM_OF]->(d)
    SET i.Occurence = toInteger($Occurence)
    ''', parameters = {'Symptom':row['Symptom'], 'Disease': row['Disease'],'Occurence': row['Count of Disease Occurrence']})
graph.commit(tx) 

ok


In [40]:
manualEntry='UMLS:C0455769_energy increased'
theCypherQuery= '''MATCH (a:SYMPTOM {Name:$queryparam})-[r]-(b)
WITH a as SYMPTOM,b as Diag, r.Occurence as relation_value
ORDER BY relation_value DESC
RETURN Diag LIMIT 1'''

cypherResult = graph.evaluate(theCypherQuery,parameters = {'queryparam':manualEntry})
print(cypherResult)

(_1770:DISEASE {Name: 'UMLS:C0011570_depression mental^UMLS:C0011581_depressive disorder'})


In [57]:
results=(cypherResult["Name"])

list_results=results.split('^')
query=[]
for result in list_results:
    tempvar=(result.split('_')[1])
    query.append(tempvar)
print(query)
for var in query:    
    query_string="https://www.merckmanuals.com/professional/SearchResults?query="
    var_list=var.split(' ')
    for wor in var_list:
        query_string=query_string+wor+"+"
    query_string=query_string+"&species="
    print(query_string)

['depression mental', 'depressive disorder']
https://www.merckmanuals.com/professional/SearchResults?query=depression+mental+&species=
https://www.merckmanuals.com/professional/SearchResults?query=depressive+disorder+&species=
